In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
from datetime import datetime, timedelta
import random

## Scrape the University of Iowa Crime Log

In [2]:
browser = webdriver.Chrome()
browser.maximize_window()

# open the Iowa City crime log page
url = 'https://safety.uiowa.edu/crime-log#accordion-item-2146-3'
browser.get(url)

# find the start date and end date input fields
start_date_input = browser.find_element(By.ID, 'StartDate')

# clear the start date and set it to the beginning of the year
start_date_input.clear()
start_date_input.send_keys('04-01')

# find search button and click
search_button = browser.find_element(By.XPATH, '//input[@type="button" and @value="Search"]')

browser.execute_script("arguments[0].click();", search_button)

# wait for results
time.sleep(5)

# find the table body
table_body = browser.find_element(By.ID, 'table-data')

# get all the rows in the table
rows = table_body.find_elements(By.TAG_NAME, 'tr')

# make an empty list to store data
crime_data = []

# loop through each row and extract the data
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')

    # check if row has 6 columns
    if len(columns) == 6:
        case_number = columns[0].text
        classification = columns[1].text
        date_time_occurred = columns[2].text
        date_time_reported = columns[3].text
        location = columns[4].text
        disposition = columns[5].text

    # append the row data to the list
    crime_data.append({
        'Case Number': case_number,
        'Classification': classification,
        'Date/Time Occurred': date_time_occurred,
        'Date/Time Reported': date_time_reported,
        'Location': location,
        'Disposition': disposition
    })

# close browser
browser.quit()

# convert into Pandas DF
crime_df = pd.DataFrame(crime_data)

# convert to CSV file
crime_df.to_csv('uiowa_city_crime_data_new.csv', index=False)

# display
print(crime_df)



      Case Number                                     Classification  \
0    CRM-2025-415                              Liquor Law Violations   
1    CRM-2025-413                                     Simple Assault   
2    CRM-2025-410                                  Simple Assault x3   
3    CRM-2025-412                                  Criminal Mischief   
4    CRM-2025-406  Disorderly - Public Urination, Liquor Law Viol...   
..            ...                                                ...   
129  CRM-2025-296                                     Simple Assault   
130  CRM-2025-294                                     Simple Assault   
131  CSA-2025-199                                         Harassment   
132  CRM-2025-291                                              Theft   
133  CRM-2025-292                                  Criminal Mischief   

                                Date/Time Occurred Date/Time Reported  \
0                                 05/07/2025 01:44   05/07/202

## Scrape the Weather

In [3]:
browser = webdriver.Chrome()
browser.maximize_window()

# open the Cedar Rapids Weather History page (Iowa City doesn't have a history option)
base_url = 'https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/'


# Set date range
start_date = datetime(2025, 3, 31) + timedelta(days=1)
end_date = datetime(2025, 5, 8)
current_date = start_date

# Data storage
weather_data = []

while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    url = base_url + date_str
    print(f"Scraping {url}")
    browser.get(url)

    try:
        # Wait for weather table to load
        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'mat-table')]"))
        )

        # Extract all table rows
        rows = browser.find_elements(By.XPATH, "//table[contains(@class, 'mat-table')]/tbody/tr")

        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            if columns:
                time_of_day = columns[0].text.strip()
                temp = columns[1].text.strip()
                dew_point = columns[2].text.strip()
                humidity = columns[3].text.strip()
                wind_direction = columns[4].text.strip()
                wind_speed = columns[5].text.strip()
                wind_gust = columns[6].text.strip()
                pressure = columns[7].text.strip()
                precip = columns[8].text.strip()
                condition = columns[9].text.strip()

                weather_data.append([date_str, time_of_day, temp, dew_point, humidity, wind_direction, wind_speed, wind_gust, pressure, precip, condition])

    except Exception as e:
        print(f"Failed to scrape {date_str}: {e}")

    current_date += timedelta(days=1)
    time.sleep(2) 

# close browser
browser.quit()

# Save to CSV
weather_df = pd.DataFrame(weather_data, columns=["Date", "Time", "Temperature", "Dew Point", "Humidity", "Wind", "Wind Speed", "Wind Gust", "Pressure", "Precip.", "Condition"])
weather_df.to_csv("cedarrapids_weather_new.csv", index=False)
print("Scraping completed. Data saved to wunderground_weather.csv")

# display
print(weather_df)
                

Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-04-01
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-04-02
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-04-03
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-04-04
Scraping https://www.wunderground.com/history/daily/us/ia/cedar-rapids/KCID/date/2025-04-05
Failed to scrape 2025-04-05: Message: unknown error: cannot determine loading status
from unknown error: missing or invalid columnNumber
  (Session info: chrome=135.0.7049.115)
Stacktrace:
	GetHandleVerifier [0x00007FF6602DEFA5+77893]
	GetHandleVerifier [0x00007FF6602DF000+77984]
	(No symbol) [0x00007FF6600A91BA]
	(No symbol) [0x00007FF66008A43C]
	(No symbol) [0x00007FF660088D0B]
	(No symbol) [0x00007FF66009673E]
	(No symbol) [0x00007FF6600978A0]
	(No symbol) [0x00007FF660096B32]
	(No symbol) [0x00007FF6600965F6]
	(No symbol) [0x00007

In [ ]:
# merge with existing data
crime_old = pd.read_csv('uiowa_city_crime_data.csv')
crime_new = pd.read_csv('uiowa_city_crime_data_new.csv')
weather_old = pd.read_csv('cedarrapids_weather.csv')
weather_new = pd.read_csv('cedarrapids_weather_new.csv')

# combine
crime_combined = pd.concat([crime_old, crime_new], ignore_index=True)
weather_combined = pd.concat([weather_old, weather_new], ignore_index=True)

# save combined versions
crime_combined.to_csv('uiowa_city_crime_data_full.csv', index=False)
weather_combined.to_csv('cedarrapids_weather_full.csv', index=False)

## Cleaning

In [ ]:
# Split the crime 'Date/Time Occurred' into Date and Time
crime_df['Date/Time Reported'] = pd.to_datetime(crime_df['Date/Time Reported'], format = '%m/%d/%Y %H:%M')
crime_df['Date'] = crime_df['Date/Time Reported'].dt.date
crime_df['Time'] = crime_df['Date/Time Reported'].dt.strftime('%H:%M')

display(crime_df)

In [ ]:
# convert 'Date' in weather data to DateTime format
weather_df['Date'] = pd.to_datetime(weather_df['Date'], format='%Y-%m-%d')
crime_df['Date'] = pd.to_datetime(crime_df['Date'])

print(weather_df)
print(crime_df)

In [ ]:
# Create new column (time buckets) (this is for future analysis)
def assign_time_bucket(time):
    if pd.to_datetime(time).hour < 6:
        return 'Night'
    elif pd.to_datetime(time).hour < 12:
        return 'Morning'
    elif pd.to_datetime(time).hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

# Apply to both datasets
merged_df['Time Bucket'] = merged_df['Time'].apply(assign_time_bucket)
crime_df['Time Bucket'] = crime_df['Time'].apply(assign_time_bucket)

print(weather_df[['Time', 'Time Bucket']])

## Merge Datasets

In [ ]:
'''
Weather data is recorded at specific time intervals (mostly on the hour),
so I'm going to match each crime's time to the nearest available weather record
for that day
'''
# convert 'Time' to datetime
crime_df['DateTime'] = pd.to_datetime(crime_df['Date'].astype(str) + ' ' + crime_df['Time'])
weather_df['DateTime'] = pd.to_datetime(weather_df['Date'].astype(str) + ' ' + weather_df['Time'])

# function to find closest available weather time for each crime entry
def find_nearest_time(crime_time, weather_times):
    return weather_times.iloc[(weather_times - crime_time).abs().argsort()[0]] # find closest time

# apply function to assign closest time from weather_df to each crime record
crime_df['Nearest Weather Time'] = crime_df['DateTime'].apply(lambda x: find_nearest_time(x, weather_df['DateTime']))

# convert back to time format
crime_df['Nearest Weather Time'] = crime_df['Nearest Weather Time'].dt.strftime('%H:%M')

# format 'Time' in weather_df
weather_df['Time'] = weather_df['DateTime'].dt.strftime('%H:%M')

# finally merge on date and nearest weather time
merged_df = crime_df.merge(weather_df, left_on=['Date', 'Nearest Weather Time'], right_on=['Date','Time'], how = 'left')

# display
display(merged_df)

In [ ]:
'''
messed up earlier by applying the time buckets to both datasets, 
undoing that here
'''

# drop duplicate columns
merged_df = merged_df.drop(columns=['Time_y', 'Time Bucket_y', 'DateTime_y'])

# fix the remaining columns' names
merged_df.rename(columns={'Time_x': 'Time', 'Time Bucket_x': 'Time Bucket', 'DateTime_x': 'DateTime'}, inplace=True)

# display cleaned up dataframe
display(merged_df)

In [ ]:
# Realized there is more cleaning to be done (should've done before merging)
merged_df['Temperature'] = merged_df['Temperature'].str.extract('([0-9.]+)').astype(float)
merged_df['Dew Point'] = merged_df['Dew Point'].str.extract('([0-9.]+)').astype(float)
merged_df['Humidity'] = merged_df['Humidity'].str.extract('([0-9.]+)').astype(float)
merged_df['Wind Speed'] = merged_df['Wind Speed'].str.extract('([0-9.]+)').astype(float)
merged_df['Wind Gust'] = merged_df['Wind Gust'].str.extract('([0-9.]+)').astype(float)
merged_df['Pressure'] = merged_df['Pressure'].str.extract('([0-9.]+)').astype(float)
merged_df['Precip.'] = merged_df['Precip.'].str.extract('([0-9.]+)').astype(float)

# display the recleaned dataframe
display(merged_df)

In [ ]:
# export the merged dataframe
merged_df.to_csv('merged_data_new.csv', index=False)